<a href="https://colab.research.google.com/github/RenatoBatista/AluraAI/blob/main/PapAI_Assistente_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-generativeai
!pip install -q google-adk
!pip install -q ipywidgets

In [ ]:
# Etapa 0: Instalar as bibliotecas necessárias
# Descomente e execute esta célula se você ainda não instalou as bibliotecas.
# !pip install -q google-generativeai
# !pip install -q google-adk
# !pip install -q ipywidgets

import os
import textwrap
from datetime import date, datetime

# Importações da biblioteca do Google
from google.colab import userdata # Para buscar a API Key no Colab
from google import genai
from google.genai import types as genai_types # Para criar Conteúdo (Content e Part)
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
# from google.adk.tools import google_search # Descomentado se quiser usar a busca

# Para exibir texto formatado e widgets no Colab
from IPython.display import display, Markdown, clear_output, HTML
import ipywidgets as widgets
import warnings

warnings.filterwarnings("ignore") # Ignorar avisos para uma saída mais limpa

# Etapa 1: Configurar a API Key do Google Gemini
# Certifique-se de ter sua GOOGLE_API_KEY armazenada nos Secrets do Colab.
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    # Configura o cliente da SDK do Gemini
    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
    print("API Key configurada com sucesso!")
except Exception as e:
    print(f"Erro ao configurar a API Key: {e}")
    print("Certifique-se de que a GOOGLE_API_KEY está definida nos 'Secrets' do Google Colab.")

# Etapa 2: Funções Auxiliares

def call_agent(agent: Agent, message_text: str) -> str:
    """
    Envia uma mensagem para um agente via Runner e retorna a resposta final.
    """
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user_pai_mae", session_id="sessao_conselho")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = genai_types.Content(role="user", parts=[genai_types.Part(text=message_text)])

    final_response = ""
    try:
        for event in runner.run(user_id="user_pai_mae", session_id="sessao_conselho", new_message=content):
            if event.is_final_response():
                for part in event.content.parts:
                    if part.text is not None:
                        final_response += part.text
                        final_response += "\n"
    except Exception as e:
        print(f"Erro ao executar o agente com ADK Runner: {e}")
        print("Tentando uma chamada direta ao modelo Gemini para este agente...")
        if hasattr(agent, 'model') and hasattr(agent, 'instruction'):
            try:
                model_instance = genai.GenerativeModel(
                    model_name=agent.model,
                    system_instruction=agent.instruction
                )
                response = model_instance.generate_content(message_text)
                final_response = response.text
            except Exception as fallback_e:
                print(f"Erro na chamada direta de fallback: {fallback_e}")
                final_response = "Desculpe, ocorreu um erro ao processar sua solicitação com o ADK e o fallback também falhou."
        else:
            final_response = "Desculpe, ocorreu um erro ao processar sua solicitação com o ADK e o fallback falhou (faltam atributos no agente)."

    return final_response.strip()


def to_markdown(text):
    """
    Formata o texto para exibição em Markdown no Colab.
    """
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


def calcular_idade_bebe(data_nascimento_str: str) -> tuple[int, int, str]:
    """
    Calcula a idade do bebê em anos, meses e retorna uma string descritiva.
    Retorna (anos, meses_totais, string_formatada_idade)
    """
    try:
        data_nascimento = datetime.strptime(data_nascimento_str, "%d/%m/%Y").date()
    except ValueError:
        return 0, 0, "Data de nascimento inválida. Use o formato DD/MM/AAAA."

    hoje = date.today()
    if data_nascimento > hoje:
        return 0,0, "Data de nascimento parece ser no futuro."

    anos = hoje.year - data_nascimento.year - ((hoje.month, hoje.day) < (data_nascimento.month, data_nascimento.day))
    meses_totais = (hoje.year - data_nascimento.year) * 12 + hoje.month - data_nascimento.month

    if hoje.day < data_nascimento.day:
        if meses_totais > 0 :
             meses_totais -=1

    if anos == 0:
        if meses_totais == 0:
            dias = (hoje - data_nascimento).days
            if dias == 0:
                return 0,0, "Recém-nascido (hoje)"
            elif dias == 1:
                 return 0, 0, f"{dias} dia de vida"
            return 0, 0, f"{dias} dias de vida"
        elif meses_totais == 1:
            return 0, 1, "1 mês"
        else:
            return 0, meses_totais, f"{meses_totais} meses"
    elif anos == 1:
        meses_do_ano = meses_totais % 12
        if meses_do_ano == 0:
             return 1, meses_totais, "1 ano"
        else:
            return 1, meses_totais, f"1 ano e {meses_do_ano} meses"
    else:
        meses_do_ano = meses_totais % 12
        return anos, meses_totais, f"{anos} anos e {meses_do_ano} meses" if meses_do_ano > 0 else f"{anos} anos"

# Etapa 3: Definições dos Agentes Especialistas

MODELO_GEMINI_PADRAO = "gemini-2.5-flash-preview-04-17"

DISCLAIMER_OBRIGATORIO = (
    "Lembre-se: Estas são dicas informativas e não substituem a avaliação e orientação de um pediatra ou profissional de saúde qualificado. "
    "Se tiver qualquer preocupação com a saúde ou o desenvolvimento do seu bebê, consulte um profissional."
)
DISCLAIMER_VACINAS_ESPECIFICO = (
    f"{DISCLAIMER_OBRIGATORIO} Para informações precisas e atualizadas sobre o calendário de vacinação, "
    "consulte o cartão de vacinação do seu bebê, o pediatra e o calendário oficial do Programa Nacional de Imunizações (PNI) do seu país."
)


def criar_instrucao_agente(nome_especialidade: str, detalhes_especialidade: str, dados_bebe_formatados: str, pergunta_pais: str) -> str:
    """Cria a instrução (system prompt) para um agente especialista."""
    return f"""
    Você é um(a) {nome_especialidade} virtual, amigável, experiente e extremamente empático(a).
    Sua principal missão é apoiar pais de primeira viagem, fornecendo informações seguras,
    práticas e personalizadas sobre o desenvolvimento e cuidados com seus bebês.

    **Especialidade do Agente:** {detalhes_especialidade}

    **Contexto do Bebê (fornecido pelo sistema):**
    {dados_bebe_formatados}

    **Tópico Principal ou Pergunta Específica dos Pais (a ser respondida ou refinada):**
    "{pergunta_pais}"

    **Suas Tarefas:**
    1.  **Analisar o Tópico/Pergunta:** Entenda a preocupação principal dos pais relacionada à sua especialidade, com base no tópico ou pergunta fornecida.
    2.  **Se o Tópico for Geral (como um título de categoria) ou uma pergunta inicial sobre o tema:** Forneça uma visão geral informativa e dicas gerais importantes para a idade do bebê sobre esse tópico. Considere as informações de rotina e ambiente fornecidas no "Contexto do Bebê" para personalizar suas dicas quando apropriado.
    3.  **Se for uma Pergunta Específica de Acompanhamento:** Responda diretamente à pergunta, utilizando o contexto do bebê (idade, rotina, ambiente) e o tema já discutido, se relevante para a pergunta.
    4.  **Considerar a Idade:** Leve em conta a idade do bebê ao formular suas sugestões.
    5.  **Fornecer Informações e Dicas (4-6 pontos principais):** Ofereça informações claras, sugestões acionáveis e seguras. Explique o porquê, se possível.
    6.  **Tom de Voz:** Use sempre um tom de voz calmo, compreensivo, positivo e encorajador.
    7.  **Segurança em Primeiro Lugar:** NÃO forneça conselhos médicos específicos ou diagnósticos. Suas dicas devem ser gerais e seguras.
    8.  **Quando Procurar um Profissional:** Se a pergunta sugerir algo que possa ser uma preocupação médica mais séria, ou se o problema persistir, gentilmente sugira que consultem um pediatra.
    9.  **Disclaimer Obrigatório:** SEMPRE finalize sua resposta EXATAMENTE com o seguinte aviso:
        "{DISCLAIMER_OBRIGATORIO}"

    **O que NÃO fazer:**
    * Não fazer promessas irrealistas.
    * Não usar linguagem julgadora.
    * Não inventar informações.
    * Não diagnosticar condições médicas.

    Responda de forma estruturada, talvez usando tópicos (bullets) para facilitar a leitura.
    """

def obter_conselho_agente_especialista(
    nome_agente: str,
    nome_especialidade_prompt: str,
    detalhes_especialidade_prompt: str,
    dados_bebe_formatados: str,
    pergunta_pais: str,
    modelo_gemini: str = MODELO_GEMINI_PADRAO
):
    """Função genérica para chamar qualquer agente especialista."""
    instrucao = criar_instrucao_agente(nome_especialidade_prompt, detalhes_especialidade_prompt, dados_bebe_formatados, pergunta_pais)
    agente = Agent(
        name=nome_agente,
        model=modelo_gemini,
        instruction=instrucao,
    )
    conselho_gerado = call_agent(agente, pergunta_pais)
    return conselho_gerado

# --- Funções específicas para cada agente ---

def obter_conselho_sono(dados_bebe_formatados: str, pergunta_pais: str):
    return obter_conselho_agente_especialista(
        nome_agente="agente_conselheiro_sono_bebe",
        nome_especialidade_prompt="Conselheiro de Sono Infantil",
        detalhes_especialidade_prompt=
        '''
        Especialista em padrões de sono, rotinas, cochilos e dificuldades comuns relacionadas ao sono do bebê.
        Dar dicas de como criar as rotinas de sono.
        Dar dicas de melhor forma de fazer o bebe dormir durante a noite e como fazer a rotina durante a madrugada.
        ''',
        dados_bebe_formatados=dados_bebe_formatados,
        pergunta_pais=pergunta_pais
    )

def obter_conselho_alimentacao(dados_bebe_formatados: str, pergunta_pais: str):
    return obter_conselho_agente_especialista(
        nome_agente="agente_conselheiro_alimentacao_bebe",
        nome_especialidade_prompt="Conselheiro de Alimentação Infantil",
        detalhes_especialidade_prompt=
        '''
        Especialista em amamentação, fórmula, introdução de alimentos sólidos, hidratação e dúvidas comuns sobre a nutrição do bebê.
        Dar dicas sobre melhores práticas de amamentação.
        ''',
        dados_bebe_formatados=dados_bebe_formatados,
        pergunta_pais=pergunta_pais
    )

def obter_conselho_cuidados_diarios(dados_bebe_formatados: str, pergunta_pais: str):
    return obter_conselho_agente_especialista(
        nome_agente="agente_guia_cuidados_diarios_bebe",
        nome_especialidade_prompt="Guia de Cuidados Diários com o Bebê",
        detalhes_especialidade_prompt=
        '''
        Especialista em banho, troca de fraldas, cuidados com a pele, coto umbilical (recém-nascidos), corte de unhas e higiene geral do bebê.
        Forneça dicas de pomadas para troca de fraldas, higiene geral e cuidados com a pele.
        Reforce a importância da segurança na hora de fazer os cuidados com o bebê.
        ''',
        dados_bebe_formatados=dados_bebe_formatados,
        pergunta_pais=pergunta_pais
    )

def obter_info_vacinas(dados_bebe_formatados: str, pergunta_pais_especifica_ou_subtema: str):
    idade_bebe_texto = "idade não especificada"
    try:
        if "Idade do Bebê:" in dados_bebe_formatados:
            idade_bebe_texto = dados_bebe_formatados.split("Idade do Bebê:")[1].split("\n")[0].strip()
    except:
        pass

    instrucao_vacinas = f"""
    Você é um Informante sobre Vacinação Infantil virtual, focado em fornecer informações gerais, seguras e empáticas.
    Sua principal missão é apoiar pais de primeira viagem a entenderem a importância da vacinação e saberem como buscar informações corretas.

    **Contexto do Bebê (fornecido pelo sistema):**
    {dados_bebe_formatados}

    **Pergunta Específica dos Pais ou Sub-tema Selecionado sobre Vacinas:**
    "{pergunta_pais_especifica_ou_subtema}"

    **Suas Tarefas (responda com base na pergunta/sub-tema):**

    * **Se a pergunta/sub-tema for "Próximas vacinas (calendário geral)":**
        * ** FORNEÇA UM CALENDÁRIO OU LISTA DE VACINAS POR NOME.** Alerte que Isso é responsabilidade do pediatra e do calendário oficial.
        * Explique que o calendário de vacinação é uma ferramenta crucial para proteger o bebê contra diversas doenças graves.
        * Mencione que, para um bebê com aproximadamente **{idade_bebe_texto}**, o calendário nacional de imunizações (como o PNI no Brasil) geralmente inclui vacinas que protegem contra doenças como [mencione 2-3 exemplos de DOENÇAS e não nomes de vacinas, ex: difteria, tétano, coqueluche, poliomielite, formas graves de tuberculose, hepatite B, rotavírus, meningites bacterianas, pneumonias]. A escolha dos exemplos deve ser genérica e ilustrativa, adequada à faixa etária geral (ex: para recém-nascidos, BCG e Hepatite B são comuns; para 2 meses, pentavalente, pólio, rotavírus, pneumo10).
        * **REFORCE FORTEMENTE:** "Para saber exatamente quais vacinas seu bebê precisa tomar e quando, é ESSENCIAL consultar o cartão de vacinação dele, conversar com o pediatra e verificar o calendário oficial do Programa Nacional de Imunizações (PNI) do seu país. Eles são as fontes mais seguras e atualizadas."
        * Sugira: "Mantenha o cartão de vacinação sempre atualizado e leve-o a todas as consultas pediátricas. Anote suas dúvidas para perguntar ao pediatra."

    * **Se a pergunta/sub-tema for "Reações comuns e cuidados":**
        * Explique que as vacinas são muito seguras e estimulam o sistema imunológico do bebê a criar defesas, e que reações leves são comuns, esperadas e geralmente de curta duração.
        * Liste reações comuns e geralmente leves
        * Forneça dicas de conforto para reações leves
        * forneça Sinais de Alerta (Quando procurar o pediatra IMEDIATAMENTE ou um serviço de emergência)

    * **Se for uma PERGUNTA ESPECÍFICA DE ACOMPANHAMENTO (diferente dos dois sub-temas acima, mas ainda sobre vacinas, ou uma pergunta específica após a descrição inicial de um sub-tema):**
        * Responda à pergunta específica da forma mais clara e útil possível, sempre dentro dos limites de um conselheiro virtual e não de um profissional de saúde.
        * Se a pergunta for sobre uma vacina específica, uma reação particular, ou um detalhe do calendário, forneça informações gerais e seguras, e sempre reforce a necessidade de discutir com o pediatra para obter orientação personalizada. Utilize o "Contexto do Bebê" (que inclui idade, rotina e ambiente, se fornecidos) caso seja relevante para a pergunta específica sobre vacinas.

    * **Tom de Voz:** Use sempre um tom de voz calmo, informativo, empático e tranquilizador.
    * **Disclaimer Obrigatório:** SEMPRE finalize sua resposta EXATAMENTE com o seguinte aviso:
        "{DISCLAIMER_VACINAS_ESPECIFICO}"

    Responda de forma clara e estruturada, usando tópicos (bullets) para facilitar a leitura das dicas e informações.
    """
    agente_vacinas = Agent(
        name="agente_informante_vacinas_bebe",
        model=MODELO_GEMINI_PADRAO,
        instruction=instrucao_vacinas,
    )
    conselho_gerado = call_agent(agente_vacinas, pergunta_pais_especifica_ou_subtema)
    return conselho_gerado


def obter_conselho_choro(dados_bebe_formatados: str, pergunta_pais: str):
    return obter_conselho_agente_especialista(
        nome_agente="agente_interprete_choro_conforto_bebe",
        nome_especialidade_prompt="Intérprete de Choro e Conselheiro de Conforto do Bebê",
        detalhes_especialidade_prompt="Ajuda a entender possíveis causas do choro (fome, sono, desconforto, dor, tédio, cólica), oferece técnicas de conforto e acalento, e orienta quando o choro persistente requer atenção pediátrica.",
        dados_bebe_formatados=dados_bebe_formatados,
        pergunta_pais=pergunta_pais
    )

def obter_guia_saltos_desenvolvimento(dados_bebe_formatados: str, pergunta_pais: str):
    return obter_conselho_agente_especialista(
        nome_agente="agente_guia_saltos_desenvolvimento_bebe",
        nome_especialidade_prompt="Guia de Saltos de Desenvolvimento do Bebê",
        detalhes_especialidade_prompt="Explica o que são os saltos de desenvolvimento, sinais comuns (irritabilidade, mudanças no sono/apetite, novas habilidades), como apoiar o bebê nessas fases e o lado positivo da aquisição de novas competências.",
        dados_bebe_formatados=dados_bebe_formatados,
        pergunta_pais=pergunta_pais
    )

def obter_conselho_geral(dados_bebe_formatados: str, pergunta_pais: str):
    return obter_conselho_agente_especialista(
        nome_agente="agente_assistente_geral_pais",
        nome_especialidade_prompt="Assistente Geral para Pais de Primeira Viagem",
        detalhes_especialidade_prompt="Fornece informações e apoio sobre uma variedade de tópicos relacionados a bebês e parentalidade não cobertos por outros especialistas. Sempre com foco na segurança e bem-estar.",
        dados_bebe_formatados=dados_bebe_formatados,
        pergunta_pais=pergunta_pais
    )

# Etapa 4: Lógica Principal do Programa Interativo com Botões

THEMES_PRINCIPAIS_MAP = {
    "Sono do Bebê": obter_conselho_sono,
    "Alimentação do Bebê": obter_conselho_alimentacao,
    "Cuidados Diários com o Bebê": obter_conselho_cuidados_diarios,
    "Vacinas do Bebê": "HANDLER_ESPECIAL_VACINAS",
    "Choro e Cólicas do Bebê": obter_conselho_choro,
    "Saltos de Desenvolvimento": obter_guia_saltos_desenvolvimento,
    "Outra Dúvida Específica": obter_conselho_geral
}
main_themes_button_descriptions = list(THEMES_PRINCIPAIS_MAP.keys())

# --- Widgets Globais ---
rotina_input_widget = widgets.Textarea(
    placeholder='(Opcional) Descreva a rotina geral do bebê...',
    layout={'width': '95%', 'height': '80px', 'margin': '5px 0px'}
)
ambiente_input_widget = widgets.Textarea(
    placeholder='(Opcional) Descreva suas principais dúvidas...',
    layout={'width': '95%', 'height': '80px', 'margin': '5px 0px'}
)
proceed_button = widgets.Button(
    description="Obter Conselho",
    button_style='success',
    layout={'width': 'auto', 'margin': '10px 0px'}
)
custom_question_input_widget = widgets.Textarea(
    placeholder='Digite sua dúvida específica aqui...',
    layout={'width': '95%', 'height': '100px', 'margin': '10px 0px'}
)
submit_custom_question_button = widgets.Button(
    description="Enviar Dúvida Específica",
    button_style='info',
    layout={'width': 'auto', 'margin': '5px 0px'}
)
follow_up_input_widget = widgets.Textarea(
    placeholder='Digite aqui sua dúvida adicional sobre o tema atual...',
    layout={'width': '95%', 'height': '80px', 'margin': '5px 0px'}
)
submit_follow_up_button = widgets.Button(
    description="Enviar Dúvida Adicional",
    button_style='primary',
    layout={'width': 'auto', 'margin': '10px 0px'}
)
btn_mais_duvidas_tema = widgets.Button(description="Mais dúvidas sobre este tema", layout={'width': 'auto', 'margin': '5px'})
btn_menu_inicial = widgets.Button(description="Ver outros temas (Menu Inicial)", layout={'width': 'auto', 'margin': '5px'})
btn_encerrar = widgets.Button(description="Encerrar Consulta", button_style='danger', layout={'width': 'auto', 'margin': '5px'})

output_area = widgets.Output()
shared_interaction_data = {
    "idade_anos": 0,
    "idade_meses_totais": 0,
    "idade_str": "Não definida",
    "current_theme_text": None,
    "current_agent_func": None,
    "dados_bebe_formatados_completos": ""
}


def assistente_pais_main():
    clear_output(wait=False)
    with output_area:
        clear_output(wait=True)
        display(HTML("<h1>🧸 Bem-vindo(a) ao Assistente IA para Pais de Primeira Viagem! O Pap<font color='red'>AI</font> 🧸</h1>"))
        display(HTML("<p>Sou um agente virtual para ajudar com dúvidas sobre bebês.</p>"))
        print("📅 Por favor, digite a data de nascimento do seu bebê e confirme.")
        print("Data de Nascimentos (DD/MM/AAAA):")
    data_nasc_input_widget = widgets.Text( layout={'width': '300px', 'margin': '0 0 10px 0'})
    confirm_date_button = widgets.Button(description="Confirmar Data", button_style='primary')

    def on_confirm_date_button_clicked(b):
        with output_area:
            clear_output(wait=True)
            data_nasc_str = data_nasc_input_widget.value
            anos, meses_totais, idade_str_calc = calcular_idade_bebe(data_nasc_str)

            shared_interaction_data["idade_anos"] = anos
            shared_interaction_data["idade_meses_totais"] = meses_totais
            shared_interaction_data["idade_str"] = idade_str_calc

            if "inválida" in idade_str_calc or "futuro" in idade_str_calc :
                print(f"Erro: {idade_str_calc}")
                print("\nPor favor, insira a data de nascimento novamente:")
                display(widgets.VBox([data_nasc_input_widget, confirm_date_button]))
                return

            print(f"✅ Seu bebê tem: {idade_str_calc}")
            display_main_theme_options()

    confirm_date_button.on_click(on_confirm_date_button_clicked)

    with output_area:
        display(widgets.VBox([data_nasc_input_widget, confirm_date_button]))

    def _get_dados_bebe_formatados_completos():
        rotina_opcional = rotina_input_widget.value
        ambiente_opcional = ambiente_input_widget.value
        #print(f"[DEBUG] Rotina lida do widget: '{rotina_opcional}'")
        #print(f"[DEBUG] Ambiente lido do widget: '{ambiente_opcional}'")
        return (
            f"Idade do Bebê: {shared_interaction_data['idade_str']} (aproximadamente {shared_interaction_data['idade_meses_totais']} meses de vida).\n"
            f"Rotina Geral Descrita pelos Pais: {rotina_opcional if rotina_opcional else 'Não informada'}.\n"
            f"Ambiente da Casa/Quarto Descrito: {ambiente_opcional if ambiente_opcional else 'Não informado'}."
        )

    def build_post_advice_options_widgets():
        """Constrói os widgets para opções pós-conselho."""
        #print("[DEBUG] build_post_advice_options_widgets() FOI CHAMADA.")

        current_theme_display_text = shared_interaction_data.get('current_theme_text', 'o tema atual')
        btn_mais_duvidas_tema.description = f"Mais dúvidas sobre '{current_theme_display_text[:25]}...'"

        post_advice_buttons_container = widgets.HBox(
            [btn_mais_duvidas_tema, btn_menu_inicial, btn_encerrar],
            layout={'justify_content': 'center', 'margin': '10px 0 0 0'}
        )

        hr_widget = widgets.HTML("<hr style='margin: 20px 0;'>") # Separador mais visível
        title_widget = widgets.HTML("<h4>O que você gostaria de fazer agora?</h4>")

        #print("[DEBUG] Botões de pós-conselho construídos.")
        return widgets.VBox([hr_widget, title_widget, post_advice_buttons_container])


    def processar_e_chamar_agente_ui(pergunta_para_agente, agente_func_para_chamar):
        shared_interaction_data["current_theme_text"] = pergunta_para_agente
        shared_interaction_data["current_agent_func"] = agente_func_para_chamar

        dados_bebe_formatados = _get_dados_bebe_formatados_completos()

        with output_area:
            clear_output(wait=True)
            print(f"Buscando informações sobre: \"{pergunta_para_agente}\". Aguarde um momento...")

            conselho_final = agente_func_para_chamar(dados_bebe_formatados, pergunta_para_agente)

            clear_output(wait=True)
            print("\n--- 📜 Conselho do Assistente IA para Pais ---")
            display(to_markdown(conselho_final))

            # Constrói e exibe os widgets de opções pós-conselho
            post_advice_widgets = build_post_advice_options_widgets()
            display(post_advice_widgets)
            #print("[DEBUG] Widgets de pós-conselho deveriam ter sido exibidos.")

    def on_submit_follow_up_button_clicked(b):
        follow_up_question = follow_up_input_widget.value
        if not follow_up_question.strip():
            with output_area:
                display(HTML(f"<h4>Mais dúvidas sobre '{shared_interaction_data['current_theme_text']}'</h4>"))
                display(HTML("<p>Por favor, digite sua dúvida adicional abaixo e clique em 'Enviar Dúvida Adicional'.</p>"))
                display(follow_up_input_widget)
                display(submit_follow_up_button)
                display(HTML("<p style='color:red;'>ERRO: Por favor, digite sua dúvida adicional.</p>"))
            return

        agente_func = shared_interaction_data["current_agent_func"]
        processar_e_chamar_agente_ui(follow_up_question, agente_func)

    submit_follow_up_button.on_click(on_submit_follow_up_button_clicked)
    btn_mais_duvidas_tema.on_click(lambda b: display_follow_up_input())


    def display_follow_up_input():
        with output_area:
            clear_output(wait=True)
            display(HTML(f"<h4>Mais dúvidas sobre '{shared_interaction_data['current_theme_text']}'</h4>"))
            display(HTML("<p>Por favor, digite sua dúvida adicional abaixo e clique em 'Enviar Dúvida Adicional'.</p>"))

            follow_up_input_widget.value = ""

            #print(f"[DEBUG] Objeto submit_follow_up_button: {submit_follow_up_button}")

            follow_up_widgets_container = widgets.VBox([
                follow_up_input_widget,
                submit_follow_up_button
            ])
            display(follow_up_widgets_container)
            #print(f"[DEBUG] follow_up_input_widget e submit_follow_up_button deveriam estar visíveis em um VBox.")


    def display_main_theme_options():
        with output_area:
            clear_output(wait=True)
            print(f"✅ Seu bebê tem: {shared_interaction_data['idade_str']}")
            print("\n--------------------------------------------------------------------")
            print("❓ Sobre qual tema principal você gostaria de conversar?")
            main_theme_buttons_widgets = create_main_theme_buttons(on_main_theme_button_clicked)
            display(widgets.VBox(main_theme_buttons_widgets))

    btn_menu_inicial.on_click(lambda b: display_main_theme_options())

    def on_encerrar_button_clicked(b):
        with output_area:
            clear_output(wait=True)
            display(HTML("<h3>Consulta Encerrada</h3>"))
            display(HTML("<p>Obrigado por usar o Assistente IA para Pais! Espero ter ajudado. 😊</p>"))
            display(HTML("<p>Lembre-se que o bem-estar do seu bebê é o mais importante. Conte sempre com o apoio do seu pediatra.</p>"))
            display(HTML("<p>Até a próxima!</p>"))
    btn_encerrar.on_click(on_encerrar_button_clicked)


    def on_submit_custom_question_button_clicked(b):
        custom_question = custom_question_input_widget.value
        if custom_question.strip():
            agente_func = THEMES_PRINCIPAIS_MAP["Outra Dúvida Específica"]
            processar_e_chamar_agente_ui(custom_question, agente_func)
        else:
            with output_area:
                clear_output(wait=True)
                print(f"✅ Seu bebê tem: {shared_interaction_data['idade_str']}")
                print(f"Tema selecionado: Outra Dúvida Específica")
                print("\nPara personalizar as dicas, conte-me um pouco mais (opcional):")
                display(rotina_input_widget, ambiente_input_widget)
                print("\nPor favor, digite sua dúvida específica abaixo e clique em 'Enviar Dúvida Específica'.")
                display(custom_question_input_widget, submit_custom_question_button)
                display(HTML("<p style='color:red;'>ERRO: O campo da dúvida não pode estar vazio.</p>"))

    submit_custom_question_button.on_click(on_submit_custom_question_button_clicked)

    def on_proceed_button_clicked(b):
        pergunta_para_agente = shared_interaction_data.get("current_theme_text")
        agente_func = shared_interaction_data.get("current_agent_func")

        if pergunta_para_agente and agente_func:
            processar_e_chamar_agente_ui(pergunta_para_agente, agente_func)
        else:
            with output_area:
                clear_output(wait=True)
                print("Erro: Não foi possível determinar o tópico ou o agente. Tente novamente.")
                display_main_theme_options()
    proceed_button.on_click(on_proceed_button_clicked)

    def on_vacinas_subtheme_button_clicked(button):
        selected_subtheme = button.description
        shared_interaction_data["current_theme_text"] = selected_subtheme
        shared_interaction_data["current_agent_func"] = obter_info_vacinas

        with output_area:
            clear_output(wait=True)
            print(f"✅ Seu bebê tem: {shared_interaction_data['idade_str']}")
            print(f"Tema: Vacinas do Bebê > {selected_subtheme}")
            print("\nPara informações mais contextualizadas, você pode preencher os campos opcionais abaixo (se ainda não o fez).")
            print("Depois, clique no botão abaixo para obter as informações.")

            rotina_input_widget.value = ""
            ambiente_input_widget.value = ""

            inputs_container = widgets.VBox([
                rotina_input_widget,
                ambiente_input_widget,
                proceed_button
            ])
            proceed_button.description = f"Obter Infos: {selected_subtheme[:25]}..."
            display(inputs_container)


    def display_vacinas_sub_options():
        with output_area:
            clear_output(wait=True)
            print(f"✅ Seu bebê tem: {shared_interaction_data['idade_str']}")
            print("Tema selecionado: Vacinas do Bebê")
            print("\nSobre qual aspecto das vacinas você gostaria de saber?")

            btn_proximas = widgets.Button(description="Próximas vacinas (calendário geral)", layout={'width': '95%', 'margin': '5px 0px'})
            btn_reacoes = widgets.Button(description="Reações comuns e cuidados", layout={'width': '95%', 'margin': '5px 0px'})

            btn_proximas.on_click(on_vacinas_subtheme_button_clicked)
            btn_reacoes.on_click(on_vacinas_subtheme_button_clicked)

            display(widgets.VBox([btn_proximas, btn_reacoes]))

    def on_main_theme_button_clicked(button):
        selected_theme = button.description
        shared_interaction_data["current_theme_text"] = selected_theme
        agente_func_ou_handler = THEMES_PRINCIPAIS_MAP.get(selected_theme)
        shared_interaction_data["current_agent_func"] = None

        with output_area:
            clear_output(wait=True)
            print(f"✅ Seu bebê tem: {shared_interaction_data['idade_str']}")
            print(f"Tema selecionado: {selected_theme}")

            rotina_input_widget.value = ""
            ambiente_input_widget.value = ""
            custom_question_input_widget.value = ""

            if agente_func_ou_handler == "HANDLER_ESPECIAL_VACINAS":
                display_vacinas_sub_options()
            elif selected_theme == "Outra Dúvida Específica":
                shared_interaction_data["current_agent_func"] = agente_func_ou_handler
                print("\nPara personalizar as dicas, conte-me um pouco mais (opcional):")
                display(rotina_input_widget)
                display(ambiente_input_widget)
                print("\nPor favor, digite sua dúvida específica abaixo e clique em 'Enviar Dúvida Específica'.")
                display(custom_question_input_widget)
                display(submit_custom_question_button)
            elif callable(agente_func_ou_handler):
                shared_interaction_data["current_agent_func"] = agente_func_ou_handler
                print("\nPara personalizar as dicas, preencha os campos abaixo (opcional).")
                print("Depois, clique em 'Obter Conselho' para prosseguir.")

                inputs_container = widgets.VBox([
                    rotina_input_widget,
                    ambiente_input_widget,
                    proceed_button
                ])
                proceed_button.description = f"Obter Conselho sobre {selected_theme[:20]}..."
                display(inputs_container)
            else:
                print(f"Desculpe, não encontrei um especialista para '{selected_theme}'.")

    def create_main_theme_buttons(handler_func):
        buttons_list = []
        for theme_desc in main_themes_button_descriptions:
            button = widgets.Button(
                description=theme_desc,
                layout={'width': '95%', 'margin': '5px 0px'}
            )
            button.on_click(handler_func)
            buttons_list.append(button)
        return buttons_list

    display(output_area) # Exibe a área de output principal que será gerenciada

# Etapa 5: Executar o assistente
if __name__ == "__main__":
    if "GOOGLE_API_KEY" not in os.environ or not os.environ["GOOGLE_API_KEY"]:
        print("A API Key do Google não foi configurada. Interrompendo a execução.")
        print("Por favor, configure a GOOGLE_API_KEY nos 'Secrets' do Colab e execute a célula de configuração novamente.")
    else:
        try:
            assistente_pais_main()
        except Exception as e:
            print(f"Ocorreu um erro inesperado durante a execução principal: {e}")

